In [2]:
from openai import OpenAI
import chromadb
from chromadb.config import Settings
from dotenv import load_dotenv
import os
import json

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OpenAI 클라이언트
client = OpenAI(api_key=OPENAI_API_KEY)

### 사용자 입력에 대해 [특허검색 / IPC / 기타 질문]으로 카테고리 선별하는 함수

In [3]:
def classify_intent_with_llm(user_input: str):
    prompt = f"""
    너는 특허 질의응답 시스템의 분류기야.
    아래 사용자 입력이 어떤 카테고리에 속하는지 하나만 답해줘.

    카테고리:
    1) PATENT_SEARCH: 비슷한 특허를 검색하거나, 기술 설명을 기반으로 유사 특허를 찾는 요청
    2) IPC_QUERY: IPC 코드(국제특허분류)에 대해 묻거나 특정 발명에 IPC가 어떻게 되는지 묻는 요청
    3) OTHER: 그 외 모든 질문

    사용자 입력:
    \"\"\"
    {user_input}
    \"\"\"

    출력 형식:
    PATENT_SEARCH 또는 IPC_QUERY 또는 OTHER
    """

    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    intent = res.choices[0].message.content.strip()
    return intent

### 사용자의 입력이 특허검색 카테고리로 판단되었으면 키워드들을 리스트로 추출하는 함수들

In [4]:
def decide_max_keywords(text: str) -> int:
    """
    입력 텍스트 길이에 따라 추출할 키워드 개수를 동적으로 결정.
    - 짧은 설명: 3개
    - 초록 + 몇 개 청구항 정도: 5개
    - 청구항 다수 포함한 긴 문서: 7개
    """
    length = len(text)

    if length < 300:      # 짧은 설명
        return 3
    elif length < 1500:   # 초록 + 몇 개 청구항
        return 5
    else:                 # 청구항 10개 이상, 긴 문서
        return 7

In [5]:
def extract_core_keywords_from_tech_doc(tech_text: str, max_keywords: int = 5) -> list[str]:
    """
    기술문서(초록 + 청구항 등)에서 특허 검색용 핵심 키워드들을 뽑아서 리스트로 반환.
    """
    prompt = f"""
    너는 특허 검색을 위한 키워드 추출기야.

    아래 기술 설명을 읽고, 이 발명의 **핵심 기술 포인트**를 {max_keywords}개 이하의 키워드로 뽑아줘.
    **핵심 기술 포인트**는 이 발명이 기존 특허들과 유사, 중복되는 점이 얼마나 있는지 검색하기 위해서 이 발명을 가장 잘 표현해줄 수 있는 핵심 키워드야.
    
    규칙:
    - "전자 장치", "터치 스크린 디스플레이", "무선 통신 회로", "센서", "메모리"처럼
    거의 모든 전자기기에 공통적으로 들어가는 일반적인 구성 요소 이름만 단독으로 쓰지 마.
    - 대신, 이 발명에서 **특히 중요한 동작 흐름/조건/조합**을 담아서 써.
    예: "결제 직전/직후 위치 데이터 + POI 서버 전송", "환경 센서 기반 결제 로그 축적" 같은 식.
    - 각 키워드는 1줄에 하나씩만 쓰고, 설명 문장 대신 **검색에 쓸 수 있는 짧은 구문** 형태로 써.
    - 가능하다면, 이 기술이 적용되는 **도메인(예: 공장 생산 라인, 의료 영상, 자율주행 등)**이 드러나도록 키워드를 작성해줘.


    기술 설명:
    \"\"\" 
    {tech_text}
    \"\"\"

    출력 형식(중요):
    - 반드시 JSON 배열 형태만 출력해.
    - 절대로 ``` 같은 마크다운 코드블록을 쓰지 마.
    - 'json'이라는 단어도 쓰지 마.
    - 예시: [
            "결제 직전 위치 데이터 + POI 서버 전송",
            "무선 결제 단말과 직접 연결된 위치 기반 결제",
            "환경 센서 기반 결제 로그 축적",
            "사용자 위치/환경 정보를 이용한 결제 기록 시스템"
            ]

    """

    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )

    raw = res.choices[0].message.content.strip()

    # JSON 형식 파싱 시도
    try:
        keywords = json.loads(raw)
        # 혹시 문자열 하나만 오거나 이상한 형식이면 보정
        if isinstance(keywords, str):
            keywords = [keywords]
        elif not isinstance(keywords, list):
            keywords = [str(keywords)]
    except json.JSONDecodeError:
        # JSON 실패 시 줄 단위 분리로 폴백
        keywords = [
            line.strip("-• ").strip()
            for line in raw.split("\n")
            if line.strip()
        ]

    # 공백 제거 및 빈 값 제거
    cleaned = [k.strip() for k in keywords if str(k).strip()]

    # max_keywords 개수 제한
    return cleaned[:max_keywords]

### 최종 함수

In [6]:
def extract_keywords_if_patent_search(user_input: str, max_keywords: int | None = None) -> dict:
    """
    1) 사용자의 입력 의도를 LLM으로 분류하고
    2) PATENT_SEARCH일 때만 기술문서에서 키워드를 추출해 반환.

    반환 형식 예시:
    - {"intent": "PATENT_SEARCH", "keywords": ["...", "..."]}
    - {"intent": "IPC_QUERY", "message": []}
    - {"intent": "OTHER", "message": []}

    max_keywords가 None이면, 입력 길이에 따라 decide_max_keywords()로 자동 결정.
    """
    intent = classify_intent_with_llm(user_input)

    if intent == "PATENT_SEARCH":
        # 길이에 따라 max_keywords 자동 결정
        if max_keywords is None:
            max_keywords = decide_max_keywords(user_input)

        keywords = extract_core_keywords_from_tech_doc(
            user_input,
            max_keywords=max_keywords
        )
        return {"intent": intent, "keywords": keywords}

    elif intent == "IPC_QUERY":
        return {"intent": "IPC_QUERY", "message": "IPC 기능은 아직 미구현입니다."}

    else:
        return {"intent": "OTHER", "message": "해당 질문은 범위 밖입니다."}


---

##### 예시 (특허검색: 긴 문장)

In [8]:
if __name__ == "__main__":
    user_input = """
    사용자 장비 (UE) 의 무선 통신의 방법으로서,챠량-대-차량 (V2V) 통신들을 위해 요구되는 적어도 하나의 자원 패턴을 결정하는 단계;기지국으로 상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하는 단계;상기 기지국으로부터, 상기 적어도 하나의 자원 패턴과 연관된 응답을 수신하는 단계로서, 상기 응답은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 응답을 수신하는 단계; 및 상기 기지국으로부터, 상기 적어도 하나의 자원 패턴에 대한 자원 할당에 대한 활성화 승인을 수신하는 단계로서, 상기 활성화 승인은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 활성화 승인을 수신하는 단계를 포함하는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>2. 제 1 항에 있어서,상기 보조 정보는 반지속적 스케줄 (SPS) 요청에 포함되는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>3. 제 1 항에 있어서,상기 보조 정보는 상기 V2V 통신들에 대한 목표 범위, 상기 V2V 통신들의 전력, 상기 V2V 통신들에 대해 요구되는 상기 적어도 하나의 자원 패턴의 주기성, 상기 V2V 통신에서의 레이턴시, 상기 V2V 통신들과 연관된 송신 타입, 시스템 프레임 넘버 (SFN) 에 대한 자원 오프셋, 또는 상기 적어도 하나의 자원 패턴이 요구되는 시간의 길이 중 하나 이상과 연관된 정보를 포함하는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>4. 제 1 항에 있어서,상기 보조 정보는 매체 액세스 제어 (MAC) 헤더 또는 무선 자원 제어 (RRC) 메시지의 제어 엘리먼트에서 전송되고,상기 적어도 하나의 자원 패턴과 연관된 상기 응답은 RRC 승인에서 수신되며,상기 활성화 승인은 데이터 채널 정보 (DCI) 승인에서 수신되는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>5. 제 1 항에 있어서,상기 적어도 하나의 자원 패턴은 복수의 자원 패턴들을 포함하고,상기 복수의 자원 패턴들의 각각의 자원 패턴은 별개의 V2V 통신과 연관되는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>6. 제 5 항에 있어서,상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하는 단계는 복수의 개개의 송신들을 전송하는 단계를 포함하고,상기 복수의 개개의 송신들의 각각의 송신은 상이한 자원 패턴과 연관되는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>7. 제 5 항에 있어서,상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하는 단계는 상기 적어도 하나의 자원 패턴과 연관된 단일의 송신을 전송하는 단계를 포함하는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>8. 제 1 항에 있어서,상기 자원 할당에 기초하여 상기 V2V 통신들을 수행하는 단계를 더 포함하는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>9. 제 1 항에 있어서,상기 기지국으로부터, 상기 자원 할당을 활성화해제하는 상기 적어도 하나의 자원 패턴과 연관된 활성화해제 승인을 수신하는 단계를 더 포함하고,상기 활성화해제 승인은 활성화해제되는 자원 패턴들의 인덱스를 포함하는, UE 의 무선 통신의 방법. </claim>
    </claimInfo>
    <claimInfo>
    <claim>10. 제 9 항에 있어서,상기 활성화 승인 또는 상기 활성화해제 승인 중 하나 이상은 다운링크 제어 정보 (DCI) 승인에서 수신되는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>11. 제 1 항에 있어서,상기 적어도 하나의 자원 패턴과 연관된 상기 응답에 포함된 무선 네트워크 임시 식별자 (RNTI) 를 사용하여 상기 활성화 승인을 프로세싱함으로써 상기 자원 할당을 결정하는 단계를 더 포함하는, UE 의 무선 통신의 방법.</claim>
    </claimInfo>
    <claimInfo>
    <claim>12. 무선 통신을 위한 장치로서,메모리; 및상기 메모리에 커플링된 적어도 하나의 프로세서를 포함하고,상기 적어도 하나의 프로세서는,챠량-대-차량 (V2V) 통신들을 위해 요구되는 적어도 하나의 자원 패턴을 결정하고;기지국으로 상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하며;상기 기지국으로부터, 상기 적어도 하나의 자원 패턴과 연관된 응답을 수신하는 것으로서, 상기 응답은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 응답을 수신하고; 및 상기 기지국으로부터, 상기 적어도 하나의 자원 패턴에 대한 자원 할당에 대한 활성화 승인을 수신하는 것으로서, 상기 활성화 승인은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 활성화 승인을 수신하도록구성된, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>13. 제 12 항에 있어서,상기 보조 정보는 반지속적 스케줄 (SPS) 요청에 포함되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>14. 제 12 항에 있어서,상기 보조 정보는 상기 V2V 통신들에 대한 목표 범위, 상기 V2V 통신들의 전력, 상기 V2V 통신들에 대해 요구되는 상기 적어도 하나의 자원 패턴의 주기성, 상기 V2V 통신에서의 레이턴시, 상기 V2V 통신들과 연관된 송신 타입, 시스템 프레임 넘버 (SFN) 에 대한 자원 오프셋, 또는 상기 적어도 하나의 자원 패턴이 요구되는 시간의 길이 중 하나 이상을 포함하는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>15. 제 12 항에 있어서,상기 보조 정보는 매체 액세스 제어 (MAC) 헤더 또는 무선 자원 제어 (RRC) 메시지의 제어 엘리먼트에서 전송되고,상기 보조 정보와 연관된 상기 응답은 RRC 승인에서 수신되며,상기 활성화 승인은 데이터 채널 정보 (DCI) 승인에서 수신되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>16. 제 12 항에 있어서,상기 적어도 하나의 자원 패턴은 복수의 자원 패턴들을 포함하고,상기 복수의 자원 패턴들의 각각의 자원 패턴은 별개의 V2V 통신과 연관되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>17. 제 12 항에 있어서,상기 적어도 하나의 프로세서는 복수의 개개의 송신들을 전송함으로써 상기 적어도 하나의 자원 패턴과 연관된 상기 보조 정보를 전송하도록 구성되고,상기 복수의 개개의 송신들의 각각의 송신은 상이한 자원 패턴과 연관되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>18. 제 12 항에 있어서,상기 적어도 하나의 프로세서는 상기 적어도 하나의 자원 패턴과 연관된 단일의 송신을 전송함으로써 상기 적어도 하나의 자원 패턴과 연관된 상기 보조 정보를 전송하도록 구성되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>19. 제 12 항에 있어서,상기 적어도 하나의 프로세서는 또한 상기 자원 할당에 기초하여 상기 V2V 통신들을 수행하도록 구성되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>20. 제 12 항에 있어서,상기 적어도 하나의 프로세서는 또한, 상기 기지국으로부터, 상기 자원 할당을 활성화해제하는 상기 적어도 하나의 자원 패턴과 연관된 활성화해제 승인을 수신하도록 구성되고,상기 활성화해제 승인은 활성화해제되는 자원 패턴들의 인덱스를 포함하는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>21. 제 20 항에 있어서,상기 활성화 승인 또는 상기 활성화해제 승인 중 하나 이상은 다운링크 제어 정보 (DCI) 승인에서 수신되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>22. 제 12 항에 있어서,상기 적어도 하나의 프로세서는 또한, 상기 적어도 하나의 자원 패턴과 연관된 상기 응답에 포함된 무선 네트워크 임시 식별자 (RNTI) 를 사용하여 상기 활성화 승인을 프로세싱함으로써 상기 자원 할당을 결정하도록 구성되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>23. 무선 통신을 위한 장치로서,챠량-대-차량 (V2V) 통신들을 위해 요구되는 적어도 하나의 자원 패턴을 결정하는 수단;기지국으로 상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하는 수단;상기 기지국으로부터, 상기 적어도 하나의 자원 패턴과 연관된 응답을 수신하는 수단으로서, 상기 응답은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 응답을 수신하는 수단; 및 상기 기지국으로부터, 상기 적어도 하나의 자원 패턴에 대한 자원 할당에 대한 활성화 승인을 수신하는 수단으로서, 상기 활성화 승인은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 활성화 승인을 수신하는 수단을 포함하는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>24. 제 23 항에 있어서,상기 보조 정보는 반지속적 스케줄 (SPS) 요청에 포함되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>25. 제 23 항에 있어서,상기 보조 정보는 상기 V2V 통신들에 대한 목표 범위, 상기 V2V 통신들의 전력, 상기 V2V 통신들에 대해 요구되는 상기 적어도 하나의 자원 패턴의 주기성, 상기 V2V 통신에서의 레이턴시, 상기 V2V 통신들과 연관된 송신 타입, 시스템 프레임 넘버 (SFN) 에 대한 자원 오프셋, 또는 상기 적어도 하나의 자원 패턴이 요구되는 시간의 길이 중 하나 이상과 연관된 정보를 포함하는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>26. 제 23 항에 있어서,상기 보조 정보는 매체 액세스 제어 (MAC) 헤더 또는 무선 자원 제어 (RRC) 메시지의 제어 엘리먼트에서 전송되고,상기 보조 정보와 연관된 상기 응답은 RRC 승인에서 수신되며,상기 활성화 승인은 데이터 채널 정보 (DCI) 승인에서 수신되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>27. 제 23 항에 있어서,상기 적어도 하나의 자원 패턴은 복수의 자원 패턴들을 포함하고,상기 복수의 자원 패턴들의 각각의 자원 패턴은 별개의 V2V 통신과 연관되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>28. 제 27 항에 있어서,상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하는 수단은 복수의 개개의 송신들을 전송하도록 구성되고,상기 복수의 개개의 송신들의 각각의 송신은 상이한 자원 패턴과 연관되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>29. 제 27 항에 있어서,상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하는 수단은 상기 적어도 하나의 자원 패턴과 연관된 정보를 포함하는 단일의 송신을 전송하도록 구성되는, 무선 통신을 위한 장치.</claim>
    </claimInfo>
    <claimInfo>
    <claim>30. 컴퓨터 실행가능 코드를 저장하는 컴퓨터 판독가능 저장 매체로서,챠량-대-차량 (V2V) 통신들을 위해 요구되는 적어도 하나의 자원 패턴을 결정하고;기지국으로 상기 적어도 하나의 자원 패턴과 연관된 보조 정보를 전송하며;상기 기지국으로부터, 상기 적어도 하나의 자원 패턴과 연관된 응답을 수신하는 것으로서, 상기 응답은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 응답을 수신하고; 및 상기 기지국으로부터, 상기 적어도 하나의 자원 패턴에 대한 자원 할당에 대한 활성화 승인을 수신하는 것으로서, 상기 활성화 승인은 승인된 자원 패턴들의 인덱스를 포함하는, 상기 활성화 승인을 수신하는코드를 포함하는, 컴퓨터 판독가능 저장 매체.</claim>

    이 특허를 출원하려고 하는데 비슷한 특허가 있는지 찾아봐줘.
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)


🔹 intent: PATENT_SEARCH
{'intent': 'PATENT_SEARCH', 'keywords': ['V2V 통신을 위한 자원 패턴 결정', '기지국 연관 보조 정보 전송', '승인된 자원 패턴 인덱스 응답 수신', '활성화 승인 포함한 자원 할당', '반지속적 스케줄 요청 포함 보조 정보', '매체 액세스 제어 및 RRC 메시지 연계', '복수 자원 패턴과 개별 V2V 통신']}


##### 예시 (특허검색: 짧은 문장)

In [9]:
if __name__ == "__main__":
    user_input = """
    본 발명은 카메라로 촬영된 영상에서 공장 라인의 제품을 실시간으로 분석하여 불량 여부를 자동 판별하는 컴퓨터 비전 장치에 관한 것이다. 합성곱 신경망(CNN) 기반 특징 추출 모듈과 이상 탐지 모듈을 포함하며, 다양한 조명 변화와 회전에 대해서도 안정적인 검출 성능을 제공한다.
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: PATENT_SEARCH
{'intent': 'PATENT_SEARCH', 'keywords': ['합성곱 신경망 기반 제품 불량 판별', '조명 변화 및 회전에 강인한 이상 탐지', '실시간 공장 라인 영상 분석']}


##### 예시 (특허검색: 매우 짧은 문장)

In [8]:
if __name__ == "__main__":
    user_input = """
    사용자가 피곤해 보이면 화면이 알아서 휘어지는 기술과 비슷한 특허가 있어?
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: PATENT_SEARCH
{'intent': 'PATENT_SEARCH', 'keywords': ['사용자 피로 상태 감지 화면 조절', '피로 상태 기반 디스플레이 변형', '인체 감지 센서 기반 화면 변화']}


---

##### 예시 (특허검색: 짧은 문장)

In [15]:
if __name__ == "__main__":
    user_input = """
    대상자의 동공 움직임과 마이크로사케이드를 활용하는 인공지능 컴퓨터비전 기반 알츠하이머 진단 시스템
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: PATENT_SEARCH
{'intent': 'PATENT_SEARCH', 'keywords': ['동공 움직임 기반 알츠하이머 진단', '마이크로사케이드 활용 인공지능 분석', '컴퓨터비전 기반 진단 시스템']}


##### 예시 (특허검색: 중간 문장)

In [16]:
if __name__ == "__main__":
    user_input = """
    본 발명은 인공지능 컴퓨터 비전을 기반으로, 대상자의 동공 움직임과 마이크로사케이드(microsaccade)를 정밀하게 분석하여 알츠하이머를 조기 진단하는 시스템에 관한 것입니다.

    이는 대상자의 미세 안구 움직임 패턴을 정량적으로 측정하고, 이를 AI가 정상인 데이터와 비교하여 초기 알츠하이머 징후를 판단하는 것을 그 특징으로 합니다.

    따라서 본 발명은 기존 검사 방식 대비 비용이 저렴하고, 검사 시간이 짧으며, 비침습적으로 진단이 가능하여 사용 편의성과 효율성을 크게 향상시키는 효과가 있습니다.
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: PATENT_SEARCH
{'intent': 'PATENT_SEARCH', 'keywords': ['인공지능 기반 동공 움직임 분석', '마이크로사케이드 정밀 분석', '미세 안구 움직임 패턴 측정', '초기 알츠하이머 조기 진단 시스템', '비침습적 알츠하이머 진단']}


##### 예시 (특허검색: 긴 문장)

In [11]:
if __name__ == "__main__":
    user_input = """
    상기 주차면 관제 방법은, 객체 검출을 수행하는 학습 모델을 이용하여, 영상 센서에 의해 취득되는 차량의 영상에서 상기 차량의 경계 영역을 생성하는 단계; 상기 주차면을 정의하는 관심 영역과 상기 차량의 경계 영역 간의 중첩도를 산출하는 단계; 점유 임계 시간 이상의 중첩 시간 동안 상기 중첩도가 기준 중첩도 이상으로 확인되는 것에 응답하여, 상기 주차면의 주차면 정보를 점유 상태로 결정하는 단계; 및 상기 주차면 정보에 기초하여 상기 주차면을 포함하는 주차장의 주차 현황 정보를 제공하는 단계를 포함한다.
    이거랑 비슷한 특허 있나 검색해줘
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: PATENT_SEARCH
{'intent': 'PATENT_SEARCH', 'keywords': ['객체 검출 학습 모델 기반 차량 경계 생성', '주차면 관심 영역과 차량 경계 영역 중첩도 산출', '점유 임계 시간 기반 주차면 정보 결정', '주차장 주차 현황 정보 제공']}


---

##### 예시 (IPC 검색)

In [10]:
if __name__ == "__main__":
    user_input = """
    본 발명의 다양한 실시예들에 따른 전자 장치는, 하우징(housing), 상기 하우징의 일부를 통하여 노출되는 터치 스크린 디스플레이(touch screen display), 상기 하우징의 내부 및/또는 일부로서 형성되는 하나 이상의 안테나들(antennas), 상기 안테나들의 적어도 하나와 전기적으로 연결된 하나 이상의 무선 통신 회로들(wireless communication circuits), 상기 전자 장치가 위치하는 환경들을 나타내는 데이터를 수집하는 하나 이상의 센서들, 상기 디스플레이, 상기 하나 이상의 통신 회로들, 및 상기 하나 이상의 센서들과 전기적으로 연결된 프로세서, 및 상기 프로세서와 전기적으로 연결된 메모리를 포함하고, 상기 메모리는, 실행 시에, 상기 프로세서가, 상기 무선 통신 회로들 또는 센서들 중 하나 이상을 이용하여, 상기 전자 장치의 위치를 적어도 일부 나타내는 데이터를 수집하고, 상기 데이터를 상기 메모리에 임시적으로 저장하고, 상기 외부 전자 장치와 직접적으로 무선 연결을 설정하는 동안, 상기 무선 통신 회로들을 중 하나를 이용하여, 선택된 거리 범위 이내에서 결제 관련 정보를 상기 외부 전자 장치로 전송하고, 상기 통신 회로들 중 적어도 하나를 통하여, 상기 결제 관련 정보와 연관된 POI(point-of-interest) 정보를 수신하거나 획득하고, 및 상기 무선 통신 회로들 중 적어도 하나를 통하여, 상기 결제 관련 정보를 전송하기 직전 또는 직후에 저장된 상기 데이터의 적어도 일부와 상기 POI 정보를 외부 서버로 전송하도록 하는 인스트럭션들을 저장하는 것을 특징으로 할 수 있다.

    이 특허를 출원하려고 하는데 ipc코드를 어떻게 하면 좋을지 알려줘.
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: IPC_QUERY
{'intent': 'IPC_QUERY', 'message': 'IPC 기능은 아직 미구현입니다.'}


---

##### 예시 (기타 질문)

In [11]:
if __name__ == "__main__":
    user_input = """
    특허 출원에 대한 전반적인 과정을 알려줘.
    """

    result = extract_keywords_if_patent_search(user_input)

    print("🔹 intent:", result["intent"])
    print(result)

🔹 intent: OTHER
{'intent': 'OTHER', 'message': '해당 질문은 범위 밖입니다.'}


---